# 1. 좋은 타구(배럴)에 대하여 정의하기
- 메이저리그에서는 배럴을 타구 속도와 발사각으로 정의를 내림

# 2.타자 10명 OPS, 장타율, 출루율 예측
- OPS = 장타율 + 출루율
- 장타율 = 1루타 + (2*2루타) + (3*3루타) + (4*홈런) / 타수[AB]
- 출루율 = (안타[HIT] + 사사구) / (타수[AB] + 사사구 + 희생플라이[SF])
- 사사구 = 볼넷[BB] + 사구[HP]

In [1]:
import pandas as pd

# 데이터 셋 파일명 설명

- 평가데이터 -> test
- 경기일정 -> game
- 타자 -> hitter
- 선수 -> player
- 타구 -> batting
- 팀 -> team

# 데이터셋 설명

In [2]:
team = pd.read_csv('data/team.csv', encoding='cp949')
team.sample(5)

# T_ID : T팀코드(선수 소속팀의 고유 코드 값)

,SEASON_ID,T_ID,T_NM
5,2016,NC,NC
58,2021,SS,삼성
44,2020,LT,롯데
59,2021,WO,키움
12,2017,KT,KT


In [3]:
game = pd.read_csv('data/2021_game.csv',encoding='cp949')
game.sample(5)

# G_ID : 경기코드
# STADIUM : 해당구장

,G_ID,GDAY_DS,VISIT_KEY,HOME_KEY,HEADER_NO,GWEEK,STADIUM,ACG
325,20210624HTKT0,20210624.0,HT,KT,0,목,수원,1
12,20210407HTWO0,20210407.0,HT,WO,0,수,고척,1
189,20210521HTSS0,20210521.0,HT,SS,0,금,대구,1
211,20210527SKKT0,20210527.0,SK,KT,0,목,수원,1
285,20210615SSOB0,20210615.0,SS,OB,0,화,잠실,1


In [4]:
hitter_2018 = pd.read_csv('data/2018_hitter.csv')
hitter_2019 = pd.read_csv('data/2019_hitter.csv')
hitter_2020 = pd.read_csv('data/2020_hitter.csv')
hitter_2021 = pd.read_csv('data/2021_hitter.csv')

hitter = pd.concat([hitter_2018, hitter_2019,
                    hitter_2020, hitter_2021])

hitter.sample(5)

# GYEAR : 연도
# PCODE : 선수코드
# GAMENUM : 경기
# PA : 타석
# AB : 타수
# BA : 타율
# HIT : 안타
# HR : 홈런
# TOTB : 루타
# SLG : 장타율
# SF : 희생플라이
# BB : 볼넷(고의4구(IB) 포함한 수치)
# KK : 삼진
# IB : 고의4구
# HP : 사구
# GD : 병살타

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,SF,BB,KK,IB,HP,GD
59,2018,62947,144,620,556,0.318,177,23,288,0.518,5,44,131,4,15,11
0,2018,60100,70,169,152,0.243,37,3,63,0.414,1,12,36,0,4,5
241,2020,76158,2,4,3,0.333,1,0,1,0.333,0,0,2,0,1,0
290,2019,79705,58,117,104,0.231,24,3,37,0.356,1,11,24,0,0,1
270,2018,79530,21,15,14,0.286,4,0,6,0.429,0,1,5,0,0,1


In [5]:
player_2018 = pd.read_csv('data/2018_player.csv', encoding='cp949')
player_2019 = pd.read_csv('data/2019_player.csv', encoding='cp949')
player_2020 = pd.read_csv('data/2020_player.csv', encoding='cp949')
player_2021 = pd.read_csv('data/2021_player.csv', encoding='cp949')

player = pd.concat([player_2018, player_2019,
                    player_2020, player_2021])

player.sample(5)

# GYEAR : 시즌
# PCODE : 선수코드
# NAME : 선수명
# T_ID : 팀코드(선수 소속팀의 고유 코드 값)
# POSITION : 포지션 투 -> 투수, 포 -> 포수, 내 -> 내야수, 외 -> 외야수
# AGE_VA : 나이
# MONEY : 연봉

,GYEAR,PCODE,NAME,T_ID,POSITION,AGE_VA,MONEY
569,2018,77446,백정현,SS,투,30,15500만원
207,2018,64760,황영국,HH,투,22,2700만원
24,2021,50360,김동혁,WO,투,19,3000만원
433,2021,67443,장지훈,SS,투,24,4200만원
581,2019,76802,김성현,SK,내,32,28000만원


In [6]:
batting_2018 = pd.read_csv('data/2018_batting.csv', encoding='cp949')
batting_2019 = pd.read_csv('data/2019_batting.csv', encoding='cp949')
batting_2020 = pd.read_csv('data/2020_batting.csv', encoding='cp949')
batting_2021 = pd.read_csv('data/2021_batting.csv', encoding='cp949')

batting = pd.concat([batting_2018, batting_2019,
                     batting_2020, batting_2021])

batting.sample(5)

# GYEAR : 연도
# G_ID : 경기코드
# PIT_ID : 타구코드
# PCODE : 선수코드
# T_ID : 타자 팀 코드
# INN : 이닝
# HIT_VEL : 타구속도(km/h)
# HIT_ANG_VER : 발사각도
# HIT_RESULT : 타석결과 - 자세한 설명은 아래에서
# PIT_VEL : 상대투수 타구구속(km/h) 
# STADIUM : 해당구장

,GYEAR,G_ID,PIT_ID,PCODE,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM
26027,2019,20190817HHWO0,190817_182727,68345,WO,1,161.13,19.4,2루타,138.92,고척
7190,2020,20200609SKLG0,200609_210526,72133,LG,8,129.24,21.1,1루타,147.49,잠실
12007,2020,20200703WOKT0,200703_184806,76313,KT,1,142.10,13.1,땅볼아웃,126.50,수원
32314,2020,20201017HTLG0,201017_192247,79109,LG,6,154.84,10.4,1루타,129.30,잠실
28412,2020,20200929SKNC0,200929_192150,62025,NC,3,116.73,33.0,플라이,141.12,창원


# HIT_RESULT : 타석결과

- 1 -> 1루타
- 2 -> 2루타
- 3 -> 3루타
- 4 -> 내야안타(1루타)
- 5 -> 땅볼아웃
- 6 -> 번트아웃
- 7 -> 번트안타
- 8 -> 병살타
- 9 -> 삼중살타
- 10 -> 야수선택
- 11 -> 인필드플라이
- 12 -> 직선타
- 13 -> 파울플라이
- 14 -> 플라이
- 15 -> 홈런
- 16 -> 희생번트
- 17 -> 희생플라이

In [7]:
test = pd.read_excel('data/test.xlsx')
display(test)
submission = test.iloc[1:11,:]
display(submission)

# submission : 제출 양식

,NO.,PCODE,OPS,장타율,출루율
0,NaN,(예시),1.09,0.68,0.417
1,1,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN
5,5,NaN,NaN,NaN,NaN
6,6,NaN,NaN,NaN,NaN
7,7,NaN,NaN,NaN,NaN
8,8,NaN,NaN,NaN,NaN
9,9,NaN,NaN,NaN,NaN


,NO.,PCODE,OPS,장타율,출루율
1,1,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN
5,5,NaN,NaN,NaN,NaN
6,6,NaN,NaN,NaN,NaN
7,7,NaN,NaN,NaN,NaN
8,8,NaN,NaN,NaN,NaN
9,9,NaN,NaN,NaN,NaN
10,10,NaN,NaN,NaN,NaN
